# 4. Assignement - HDR, response function estimation and HDR composition by Debevec method

In [ ]:
import exif
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
from scipy.sparse.linalg import lsqr
from copy import copy
import requests
import os
import shutil
import tempfile


In [ ]:
def download_images(urls, path):
    for idx in range(len(urls)):
        print(urls[idx], end='')
        r = requests.get(urls[idx], stream=True)
        if r.status_code == 200:
            with open(os.path.join(path, f'image{idx:02d}.jpg'), 'bw') as f:
                # r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)    
                print('...OK')
        else:
                print('...FAIL')

def read_images(file_pattern, scale_percent = 100):
    files = glob.glob(file_pattern) 

    imgs = []
    t = []

    for file in files:
        tmp_img = cv2.imread(file) 
        width = int(tmp_img.shape[1] * scale_percent // 100) 
        height = int(tmp_img.shape[0] * scale_percent // 100)
        dim = (width, height) 
        imgs.append(cv2.cvtColor(cv2.resize(tmp_img, dim, interpolation = cv2.INTER_AREA), cv2.COLOR_BGR2RGB)) 
        info = exif.Image(file)
        t.append(info.exposure_time) 
    return (height, width), np.array(imgs), t 

def get_weights(Z, L):
    return np.interp(Z, [0, (L-1)/2, L-1], [0, 1, 0]).flatten() 


In [ ]:
images_urls = ['https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka02.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka03.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka04.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka05.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka06.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka07.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka08.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka09.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka10.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka11.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka12.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka13.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka14.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka15.jpg?raw=true',
              ]

with tempfile.TemporaryDirectory() as tempdir:
    download_images(images_urls, tempdir)
    dim, Z, t = read_images(os.path.join(tempdir, "*.jpg"), scale_percent = 1)
    
print(f'Image shape: {dim}')
print(f'Exposure times [s]: {t}')

r = Z[:, :, :, 0].flatten() 
g = Z[:, :, :, 1].flatten() 
b = Z[:, :, :, 2].flatten() 

L = 2**8
lambda_ = 2 

w_r = get_weights(r, L) 
w_g = get_weights(g, L) 
w_b = get_weights(b, L) 

t_ind, Z_ind = np.indices((Z.shape[0], Z.shape[1]*Z.shape[2]))
t_ind = t_ind.flatten()
E_ind = Z_ind

t_vec_r = np.log(np.take(t, t_ind)) * w_r
t_vec_g = np.log(np.take(t, t_ind)) * w_g
t_vec_b = np.log(np.take(t, t_ind)) * w_b

g_vec_r = np.log(r+0.01)
g_vec_g = np.log(g+0.01)
g_vec_b = np.log(b+0.01)


In [ ]:
def estimate_exposure(w, g_vec, t_ind, E_ind):
    A = np.zeros(((len(g_vec)+1), np.max(E_ind)+1+np.max(t_ind)))
    b = np.zeros((len(g_vec)+1))
    E_ind = E_ind.flatten()
    r_ind = list(range(A.shape[0]-1))
    A[r_ind, E_ind.flatten()] = np.sqrt(w)
    A[r_ind, t_ind.flatten()+np.max(E_ind)] = np.sqrt(w)
    b[:-1] = g_vec*np.sqrt(w)
    A[-1, np.max(E_ind)+1] = 1
    sol = lsqr(A, b)
    return sol[0][:np.max(E_ind)+1], sol[0][np.max(E_ind)+1:]


In [ ]:
E_exposure_r, t_est_r = estimate_exposure(w_r, g_vec_r, t_ind, E_ind)
E_exposure_g, t_est_g = estimate_exposure(w_g, g_vec_g, t_ind, E_ind)
E_exposure_b, t_est_b = estimate_exposure(w_b, g_vec_b, t_ind, E_ind)

In [ ]:
print(f'Exposure time estimation (red): ', np.exp(t_est_r))
print(f'Exposure time estimation (green): ', np.exp(t_est_g))
print(f'Exposure time estimation (blue): ', np.exp(t_est_b))
print(f'Real exposure time: ', t)

In [ ]:
def estimate_response_debevec(w, L, Z_vec, t_vec, E_ind, lambd):
    # YOUR estimate_response_debevec LOGIC HERE


In [ ]:
lambda_ = 2
g_r, _ = estimate_response_debevec(w_r, 2**8, r, t, Z_ind, lambda_)
g_g, _ = estimate_response_debevec(w_g, 2**8, g, t, Z_ind, lambda_)
g_b, _ = estimate_response_debevec(w_b, 2**8, b, t, Z_ind, lambda_)
plt.plot(g_r, 'r')
plt.plot(g_g, 'g')
plt.plot(g_b, 'b')
plt.title('Estimated camera transfer functions');
plt.xlabel('Z')
plt.ylabel('g()');

In [ ]:
def fit_response(L, g, degree=2):
    g_val = []
    for idx in range(L):
        g_val.append([idx, g[idx]])
    g_val = np.array(g_val)
    
    poly_transform = PolynomialFeatures(degree=degree, include_bias=False).fit(g_val[:,0].reshape((-1,1)))
    poly_trans = lambda x: poly_transform.transform(x.reshape((-1,1)))
    z_poly = poly_trans(g_val[:,0])
    g_model = LinearRegression()
    g_model.fit(z_poly, g_val[:,1])
    g_model_pred = g_model.predict(poly_trans(g_val[:,0]))

    plt.figure()
    plt.scatter(x=g_val[:,0], y=g_val[:,1])
    plt.plot(g_model_pred, 'r')
    
    return g_model_pred

g_model_r = fit_response(2**8, g_r, degree=2)
plt.title('Camera response function - red chanel')
g_model_g = fit_response(2**8, g_g, degree=2)
plt.title('Camera response function - green chanel')
g_model_b = fit_response(2**8, g_b, degree=2)
plt.title('Camera response function - blue chanel');

In [ ]:
with tempfile.TemporaryDirectory() as tempdir:
    download_images(images_urls, tempdir)
    dim_f, Z_f, t_f = read_images(os.path.join(tempdir, "*.jpg"), scale_percent = 100)

In [ ]:
def hdr_from_exposure(g, w, Z, t):
    w_img = np.sqrt(w.reshape((len(t),-1)))
    Z_img = np.interp(Z, list(range(2**8)), g).reshape((len(t),-1))
    E = np.zeros(w_img.shape[1], dtype=np.float32)
    t = np.log(t)
    for idx in range(len(E)):
        E[idx] = # YOUR pixel irradiance LOGIC HERE
    return E

r_f = Z_f[:, :, :, 0].flatten() 
g_f = Z_f[:, :, :, 1].flatten() 
b_f = Z_f[:, :, :, 2].flatten() 
w_r_f = get_weights(r_f, L) 
w_g_f = get_weights(g_f, L) 
w_b_f = get_weights(b_f, L) 

E_r_est = hdr_from_exposure(g_model_r, w_r_f, r_f, t)
E_g_est = hdr_from_exposure(g_model_g, w_g_f, g_f, t)
E_b_est = hdr_from_exposure(g_model_b, w_b_f, b_f, t)


In [ ]:
plt.imshow(E_r_est.reshape(dim_f))
plt.colorbar()
plt.title('Irradiance map - red chanel');

In [ ]:
plt.imshow(E_g_est.reshape(dim_f))
plt.colorbar()
plt.title('Irradiance map - green chanel');

In [ ]:
plt.imshow(E_b_est.reshape(dim_f))
plt.colorbar()
plt.title('Irradiance map - blue chanel');

In [ ]:
hdr = np.stack((E_r_est.reshape(dim_f), E_g_est.reshape(dim_f), E_b_est.reshape(dim_f)), axis=-1)
plt.imshow(hdr)
plt.title('HDR Image - clipped');

In [ ]:
tonemap1 = cv2.createTonemap(gamma=1)
res_debevec = tonemap1.process(hdr.copy())
res_debevec = np.clip(res_debevec*255, 0, 255).astype('uint8')
plt.imshow(res_debevec)
plt.title('Generated LDR image by tonemapping');